In [12]:
# Import statements
import numpy as np
import random as rand
import os
import pandas
import time

In [13]:
# This cell contains all Connect4 related functions.

# This is a wrapper class that holds the logic engine and returns a decision.
class logic_engine_wrapper:

    # Defaults logic_engine to 0 so that it will just move over one space each turn.
    def __init__(self,logic_engine=0):
        self.logic_engine = logic_engine

    # Returns the logic_engine's move based on the provided board.
    def get_move(self,board):
        if type(self.logic_engine) is int:
            if self.logic_engine >= 6:
                self.logic_engine = 0
                return 6
            else:
                self.logic_engine += 1
                return self.logic_engine - 1
        else:
            output = self.logic_engine.get_output_vector(board.get_board_vector())
            greatest = 0
            for i in range(len(output)):
                if output[i] > output[greatest]:
                    greatest = i
            return greatest
                    
# This class represents a Connect4 board and contains all necessary utilities for it.
# On boards player1 = 1, player2 = -1, and blank = 0.
# The higher the first index the higher on the board i.e. index [5][6] is the top right corner of the board.
class game_board:

    # Creates a 7 x 6 game board.
    def __init__(self):
        self.board = [[0.0 for _ in range(7)] for _ in range(6)]

    # Prints the current game board.
    def print_board(self,player1_symbol='@',player2_symbol='#',clear=False):
        if clear:
            os.system('cls')
        for row in self.board[::-1]:
            print('|',end='')
            for num in row:
                symbol = player1_symbol if num == 1 else (player2_symbol if num == -1 else ' ')
                print(symbol,end='')
            print('|')
        print('---------')
        print(' 1234567 ')

    # Drops a piece into the indicated slot using a range of [0,6]
    def drop_piece(self,slot,player):
        if slot > 6 or slot < 0:
            return False
        else:
            for i in range(6):
                if self.board[i][slot] == 0:
                    self.board[i][slot] = player
                    return True
            return False

    # Returns the board squashed into a numpy array. (This is a utility function for net use)
    def get_board_vector(self):
        board_vector = []
        for row in self.board:
            board_vector += row
        return np.array(board_vector)

    # Returns a player if one of them as won otherwise returns 0.
    def get_winner(self):
        # Checks rows.
        for row in self.board:
            previous = None
            count = 0
            for num in row:
                if num == previous:
                    count += 1
                else:
                    previous = num
                    count = 1
                if count >= 4 and previous != 0:
                    return previous
        # Checks columns.
        for x in range(7):
            previous = None
            count = 0
            for y in range(6):
                num = self.board[y][x]
                if num == previous:
                    count += 1
                else:
                    previous = num
                    count = 1
                if count >= 4 and previous != 0:
                    return previous
        # Checks diagonals.
        for y in range(6):
            for x in range(7):
                num = self.board[y][x]
                if num != 0 and y < 3:
                    if x < 4:
                        if num == self.board[y+1][x+1] and num == self.board[y+2][x+2] and num == self.board[y+3][x+3]:
                            return num
                    elif x > 2:
                        if num == self.board[y+1][x-1] and num == self.board[y+2][x-2] and num == self.board[y+3][x-3]:
                            return num

        return 0


# This function enables the user to play against a logic_engine or lets two logic_engines play against each other.
def play_game(logic_engines=[logic_engine_wrapper()],verbose=True,ai_delay=0.2):
    board = game_board()
    current_player = 1 if rand.choice([True,False]) else -1
    max_num_of_turns = 6 * 7
    for turn in range(max_num_of_turns):
        if verbose:
            board.print_board(clear=True)
        if len(logic_engines) == 2:
            if verbose:
                print("Thinking...")
            time.sleep(ai_delay)
            slot = None
            if current_player == 1:
                proxy_board = game_board()
                for y in range(len(proxy_board.board)):
                    for x in range(len(proxy_board.board[y])):
                        proxy_board.board[y][x] = -1 * board.board[y][x]
                slot = logic_engines[0].get_move(proxy_board)
            else:
                slot = logic_engines[1].get_move(board)
            board.drop_piece(slot,current_player)
        else:
            if current_player == 1:
                slot = int(input('Input a slot: ')) - 1
                board.drop_piece(slot,current_player)
            else:
                if verbose:
                    print("Thinking...")
                time.sleep(ai_delay)
                slot = logic_engines[0].get_move(board)
                board.drop_piece(slot,current_player)
        winner = board.get_winner()
        if winner != 0:
            if verbose:
                board.print_board(clear=True)
            return winner
        else:
            current_player = 1 if current_player == -1 else -1
        if turn == (6 * 7) - 1 and verbose:
            board.print_board(clear=True)
    return 0

In [14]:
#Non-linear functions are declared in this cell.

#Sigmoid function.
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [15]:
# Info String Seperators:
net_section_seperator = '|'
net_sub_section_seperator = ':' # i.e. each hidden layer and each bias layer
net_matrix_row_seperator = '>'
net_vector_index_seperator = ','

In [16]:
# Feedforward network class that's used.
class net:
    
    # wiki_net class constructor.
    def __init__(self,input_size,hidden_layers,non_linears=sigmoid,use_biases=True,name="N/A"):
        self.hidden_layers = []
        self.non_linears = []
        self.biases = []
        self.name = name
        if type(non_linears) != list:
            non_linears = [non_linears] * len(hidden_layers) 
            previous_size = input_size
        elif len(non_linears) == 1:
            non_linears *= len(hidden_layers)
        for i in range(len(hidden_layers)):
            self.hidden_layers.append(2 * np.random.random((previous_size,hidden_layers[i])) - 1)
            self.non_linears.append(non_linears[i])
            self.biases.append(2 * np.random.random(hidden_layers[i]) - 1 if use_biases else np.zeros(hidden_layers[i]))
            previous_size = hidden_layers[i]
    
    # Gets the output vector for a given input vector.
    def get_output_vector(self,input_vector):
        output_vector = input_vector
        for i in range(len(self.hidden_layers)):
            output_vector = self.non_linears[i](np.dot(output_vector,self.hidden_layers[i] + self.biases[i]))
        return output_vector
    
    # Returns a string that contains all of the network's features in the following order, [Name, Score, Hidden_Layers, Biases]
    def generate_info_string(self,score=0):
        info_string = f'{self.name}{net_section_seperator}{score}{net_section_seperator}'
        count_1 = 0
        count_2 = 0
        count_3 = 0
        for layer in self.hidden_layers:
            for vector in layer:
                for index in vector:
                    info_string += f'{index}'
                    info_string += net_vector_index_seperator if count_1 != len(vector) - 1 else ''
                    count_1 += 1
                count_1 = 0
                info_string += net_matrix_row_seperator if count_2 != len(layer) - 1 else ''
                count_2 += 1
            count_2 = 0
            info_string += net_sub_section_seperator if count_3 != len(self.hidden_layers) - 1 else ''
            count_3 += 1
        info_string += net_section_seperator
        count_1 = 0
        count_2 = 0
        for bias in self.biases:
            for index in bias:
                info_string += f'{index}'
                info_string += net_vector_index_seperator if count_1 != len(bias) - 1 else ''
                count_1 += 1
            count_1 = 0
            info_string += net_sub_section_seperator if count_2 != len(self.biases) - 1 else ''
            count_2 += 1
        return info_string
    
    # Loads the net's attribute's from a given string and returns it's score.
    # String Format is listed above the generate_info_string method
    def load_info_from_string(self,info_string,non_linear=sigmoid):
        sections = info_string.split(net_section_seperator)
        self.name = sections[0]
        score = float(sections[1])
        self.hidden_layers = []
        self.biases = []
        self.non_linears = [non_linear]
        hidden_layer_sections = sections[2].split(net_sub_section_seperator)
        for layer in hidden_layer_sections:
            vectors = []
            for text_vector in layer.split(net_matrix_row_seperator):
                vector = []
                for index in text_vector.split(net_vector_index_seperator):
                    vector.append(float(index))
                vectors.append(vector)
            self.hidden_layers.append(np.array(vectors))
        self.non_linears *= len(self.hidden_layers)
        text_biases = sections[3].split(net_sub_section_seperator)
        for text_bias in text_biases:
            bias = []
            for index in text_bias.split(net_vector_index_seperator):
                bias.append(float(index))
            self.biases.append(np.array(bias))
        return score

In [17]:
# All functions related to saving and loading data are in this cell.

# Saves the given wrapped net population.
def save_wrapped_net_population(population,name,generation):
    try:
        file = open(f'Populations/{name}.snp','w')
        file.write(f'Generation {generation}')
        for wrapped_net in population:
            file.write(f'\n{wrapped_net[0].generate_info_string(wrapped_net[1])}')
        file.close()
    except:
        print(f'Failed to save population : {name}')
        
# Loads a net population from a given file in format (population,generation)
def load_wrapped_net_population(name):
    file = None
    try:
        file = open(f'Populations/{name}.snp','r')
    except:
        print(f'Failed to find file for population : {name}')
        return None
    if file != None:
        contents = file.read().split('\n')
        generation = int(contents[0].split(' ')[1])
        population = []
        for val in contents[1:]:
            net = generate_standard_net()
            score = net.load_info_from_string(val)
            population.append((net,score))
        return (population,generation)

In [18]:
# Generates a net_net with preset values (this is basically a way to make net generation easier).
def generate_standard_net(name="N/A"): 
    return net(input_size=42,hidden_layers=[7],name=name)
    #return net(input_size=42,hidden_layers=[20,10,7],name=name)
    #return net_net(input_size=5,hidden_layers=[3,2,1],name=name)

In [19]:
# Genetic evolution related functions are declared in this cell.

# Breeds two nets using indexed addition and the specified weight, mates child with a random net if mutating.
def breed_nets(net_1,net_2,mutation=False,non_linear=sigmoid,name="N/A"):
    hidden_layers = []
    biases = []
    non_linears = []
    for i in range(len(net_1.hidden_layers)):
        #hidden_layers.append((net_1.hidden_layers[i] * net_1_weight) + (net_2.hidden_layers[i] * (1 - net_1_weight)))
        #biases.append((net_1.biases[i] * net_1_weight) + (net_2.biases[i] * (1 - net_1_weight)))
        non_linears.append(non_linear)
        hidden_layers.append([])
        biases.append([])
        for j in range(len(net_1.hidden_layers[i])):
            hidden_layers[i].append((rand.choice([net_1,net_2])).hidden_layers[i][j])
        hidden_layers[i] = np.array(hidden_layers[i])
        for j in range(len(net_1.biases[i])):
            biases[i].append((rand.choice([net_1,net_2])).biases[i][j])
        biases[i] = np.array(biases[i])
    child = generate_standard_net(name=name)
    child.hidden_layers = hidden_layers
    child.biases = biases
    child.non_linears = non_linears
    if mutation:
        return breed_nets(child,generate_standard_net(),name=name)
    else:
        return child

# Breeds a population of ordered wrapped nets.
def breed_wrapped_net_popultation(old_population,generation):
    avg_score = 0
    old_size = len(old_population)
    for wrapped_net in old_population:
        avg_score += wrapped_net[1]
    avg_score /= len(old_population)
    new_population = []
    rand.shuffle(old_population)
    for wrapped_net in old_population:
        if wrapped_net[1] >= avg_score and len(new_population) < len(old_population) / 2:
            new_population.append(wrapped_net)
    old_population = new_population
    new_population = []
    child_num = 0
    rand.shuffle(old_population)
    while len(old_population) > 1:
        wrapped_nets = [old_population.pop(),old_population.pop()]
        score_sum = wrapped_nets[0][1] + wrapped_nets[1][1]
        child_num += 1
        child_1 = breed_nets(wrapped_nets[0][0],wrapped_nets[1][0],mutation=np.random.random(1)[0] > 0.95,name=f'Net : {child_num} (Generation : {generation})')
        child_num += 1
        child_2 = breed_nets(wrapped_nets[0][0],wrapped_nets[1][0],mutation=np.random.random(1)[0] > 0.95,name=f'Net : {child_num} (Generation : {generation})')
        new_population.append((child_1,0))
        new_population.append((child_2,0))
        if avg_score <= 0:
            if np.random.random(1)[0] > 0.5:
                new_population = add_wrapped_net(new_population,wrapped_nets[0])
            else:
                new_population = add_wrapped_net(new_population,wrapped_nets[1])
        else:
            for wrapped_net in wrapped_nets:
                if wrapped_net[1] >= avg_score:
                        new_population = add_wrapped_net(new_population,wrapped_net)
    if len(old_population) <= 3:
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    while len(old_population) > 0:
        new_population = add_wrapped_net(new_population,old_population.pop())
    if len(new_population) <= 0:
        for _ in range(8):
            child_num += 1
            new_population.append((generate_standard_net_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    while len(new_population) < old_size:
        child_num += 1
        new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
   # if generation % 4:
           # child_num += 1
           # new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    if generation % 200 == 0:
        new_population = new_population[:20]
        for _ in range(25):
            child_num += 1
            new_population.append((generate_standard_net(name=f'Net : {child_num} (Generation : {generation})'),0))
    return new_population
    
# Subjects a population to natural selection for the given number of generations.
def subject_popultaion_to_natural_selection(population,starting_generation,name,num_of_generations=100,verbose_crawl=False):
    for i in range(num_of_generations):
        current_generation = starting_generation + i
        print(f'Beginning Natural Selection For Generation : {current_generation} | Population Size : {len(population)}')
        new_population = []
        high_score = 0
        count = 0
        start = time.time()
        for wrapped_net in population:
            new_population.append((wrapped_net[0],0))
        for i in range(len(new_population) - 1):
            for j in range(len(new_population) - i):
                if j != 0:
                    logic_engine_1 = logic_engine_wrapper(logic_engine=population[i][0])
                    logic_engine_2 = logic_engine_wrapper(logic_engine=population[i+j][0])
                    result = play_game(logic_engines=[logic_engine_1,logic_engine_2],verbose=False,ai_delay=0)
                    if result == 1:
                        new_population[i] = (new_population[i][0],new_population[i][1]+1)
                    elif result == -1:
                        new_population[i] = (new_population[i+j][0],new_population[i+j][1]+1)
        proxy_pop = []
        for wrapped_net in new_population:
            add_wrapped_net(proxy_pop,wrapped_net)
            if wrapped_net[1] > high_score:
                high_score = wrapped_net[1]
        new_population = proxy_pop
        end = time.time()
        print(f'Generation {current_generation} High Score : {high_score} | Duration : {end - start} Secs')
        population = breed_wrapped_net_popultation(new_population,current_generation + 1)
        save_wrapped_net_population(population,name,current_generation + 1)
        print(f'Generation {current_generation + 1} Bred')

# Generates a starting population of nets. The nets are in a tuple wrapper of format (net,score)
def generate_base_wrapped_net_population(size=100):
    return [(generate_standard_net(f'Net : {i + 1} (Generation : 0)'),0) for i in range(size)]

# Adds a net wrapper to the given array in order of greatest score.
def add_wrapped_net(array,wrapped_net):
    if len(array) == 0:
        array.append(wrapped_net)
    else:
        placed = False
        for i in range(len(array)):
            if array[i][1] < wrapped_net[1] and not placed:
                array.insert(i,wrapped_net)
                placed = True
        if not placed:
            array.append(wrapped_net)
    return array

In [20]:
# Creates a new population.
population = generate_base_wrapped_net_population(10)
generation = 0

In [21]:
# Loads a population from the disk.
hold = load_wrapped_net_population('eigthTrial')
population = hold[0]
generation = hold[1]

In [ ]:
subject_popultaion_to_natural_selection(population,generation,name='eigthTrial',num_of_generations=50000,verbose_crawl=False)

Beginning Natural Selection For Generation : 25143 | Population Size : 107
Generation 25143 High Score : 6 | Duration : 3.4323604106903076 Secs
Generation 25144 Bred
Beginning Natural Selection For Generation : 25144 | Population Size : 107
Generation 25144 High Score : 18 | Duration : 2.794506788253784 Secs
Generation 25145 Bred
Beginning Natural Selection For Generation : 25145 | Population Size : 111
Generation 25145 High Score : 14 | Duration : 4.033498287200928 Secs
Generation 25146 Bred
Beginning Natural Selection For Generation : 25146 | Population Size : 111
Generation 25146 High Score : 10 | Duration : 3.7953875064849854 Secs
Generation 25147 Bred
Beginning Natural Selection For Generation : 25147 | Population Size : 111
Generation 25147 High Score : 8 | Duration : 3.3484904766082764 Secs
Generation 25148 Bred
Beginning Natural Selection For Generation : 25148 | Population Size : 111
Generation 25148 High Score : 9 | Duration : 3.7815322875976562 Secs
Generation 25149 Bred
Beg

Generation 25193 High Score : 11 | Duration : 5.0795698165893555 Secs
Generation 25194 Bred
Beginning Natural Selection For Generation : 25194 | Population Size : 131
Generation 25194 High Score : 8 | Duration : 5.159421443939209 Secs
Generation 25195 Bred
Beginning Natural Selection For Generation : 25195 | Population Size : 135
Generation 25195 High Score : 11 | Duration : 4.446139097213745 Secs
Generation 25196 Bred
Beginning Natural Selection For Generation : 25196 | Population Size : 139
Generation 25196 High Score : 5 | Duration : 4.082076072692871 Secs
Generation 25197 Bred
Beginning Natural Selection For Generation : 25197 | Population Size : 139
Generation 25197 High Score : 9 | Duration : 6.701974153518677 Secs
Generation 25198 Bred
Beginning Natural Selection For Generation : 25198 | Population Size : 139
Generation 25198 High Score : 9 | Duration : 6.660318374633789 Secs
Generation 25199 Bred
Beginning Natural Selection For Generation : 25199 | Population Size : 139
Generat

Generation 25243 High Score : 9 | Duration : 1.0480895042419434 Secs
Generation 25244 Bred
Beginning Natural Selection For Generation : 25244 | Population Size : 67
Generation 25244 High Score : 6 | Duration : 1.4478209018707275 Secs
Generation 25245 Bred
Beginning Natural Selection For Generation : 25245 | Population Size : 67
Generation 25245 High Score : 7 | Duration : 1.428523302078247 Secs
Generation 25246 Bred
Beginning Natural Selection For Generation : 25246 | Population Size : 67
Generation 25246 High Score : 6 | Duration : 1.5287086963653564 Secs
Generation 25247 Bred
Beginning Natural Selection For Generation : 25247 | Population Size : 67
Generation 25247 High Score : 9 | Duration : 1.3442072868347168 Secs
Generation 25248 Bred
Beginning Natural Selection For Generation : 25248 | Population Size : 67
Generation 25248 High Score : 10 | Duration : 1.5356552600860596 Secs
Generation 25249 Bred
Beginning Natural Selection For Generation : 25249 | Population Size : 67
Generation

Generation 25293 High Score : 10 | Duration : 2.6719894409179688 Secs
Generation 25294 Bred
Beginning Natural Selection For Generation : 25294 | Population Size : 95
Generation 25294 High Score : 8 | Duration : 2.8867602348327637 Secs
Generation 25295 Bred
Beginning Natural Selection For Generation : 25295 | Population Size : 95
Generation 25295 High Score : 9 | Duration : 2.609452486038208 Secs
Generation 25296 Bred
Beginning Natural Selection For Generation : 25296 | Population Size : 95
Generation 25296 High Score : 10 | Duration : 2.7265539169311523 Secs
Generation 25297 Bred
Beginning Natural Selection For Generation : 25297 | Population Size : 95
Generation 25297 High Score : 7 | Duration : 2.0951006412506104 Secs
Generation 25298 Bred
Beginning Natural Selection For Generation : 25298 | Population Size : 95
Generation 25298 High Score : 9 | Duration : 3.2994301319122314 Secs
Generation 25299 Bred
Beginning Natural Selection For Generation : 25299 | Population Size : 95
Generatio

Generation 25343 High Score : 9 | Duration : 3.985893964767456 Secs
Generation 25344 Bred
Beginning Natural Selection For Generation : 25344 | Population Size : 119
Generation 25344 High Score : 9 | Duration : 4.692155599594116 Secs
Generation 25345 Bred
Beginning Natural Selection For Generation : 25345 | Population Size : 119
Generation 25345 High Score : 8 | Duration : 4.271090745925903 Secs
Generation 25346 Bred
Beginning Natural Selection For Generation : 25346 | Population Size : 119
Generation 25346 High Score : 9 | Duration : 4.391540288925171 Secs
Generation 25347 Bred
Beginning Natural Selection For Generation : 25347 | Population Size : 119
Generation 25347 High Score : 6 | Duration : 4.562737703323364 Secs
Generation 25348 Bred
Beginning Natural Selection For Generation : 25348 | Population Size : 119
Generation 25348 High Score : 8 | Duration : 4.502681493759155 Secs
Generation 25349 Bred
Beginning Natural Selection For Generation : 25349 | Population Size : 119
Generation

Generation 25393 High Score : 15 | Duration : 5.165873765945435 Secs
Generation 25394 Bred
Beginning Natural Selection For Generation : 25394 | Population Size : 139
Generation 25394 High Score : 7 | Duration : 5.231304883956909 Secs
Generation 25395 Bred
Beginning Natural Selection For Generation : 25395 | Population Size : 139
Generation 25395 High Score : 11 | Duration : 5.939136743545532 Secs
Generation 25396 Bred
Beginning Natural Selection For Generation : 25396 | Population Size : 139
Generation 25396 High Score : 11 | Duration : 6.102777004241943 Secs
Generation 25397 Bred
Beginning Natural Selection For Generation : 25397 | Population Size : 139
Generation 25397 High Score : 9 | Duration : 5.991217613220215 Secs
Generation 25398 Bred
Beginning Natural Selection For Generation : 25398 | Population Size : 139
Generation 25398 High Score : 7 | Duration : 5.704031944274902 Secs
Generation 25399 Bred
Beginning Natural Selection For Generation : 25399 | Population Size : 139
Generat

Generation 25443 High Score : 7 | Duration : 1.721116065979004 Secs
Generation 25444 Bred
Beginning Natural Selection For Generation : 25444 | Population Size : 75
Generation 25444 High Score : 7 | Duration : 1.661142349243164 Secs
Generation 25445 Bred
Beginning Natural Selection For Generation : 25445 | Population Size : 75
Generation 25445 High Score : 12 | Duration : 1.7151634693145752 Secs
Generation 25446 Bred
Beginning Natural Selection For Generation : 25446 | Population Size : 75
Generation 25446 High Score : 6 | Duration : 1.8818635940551758 Secs
Generation 25447 Bred
Beginning Natural Selection For Generation : 25447 | Population Size : 75
Generation 25447 High Score : 6 | Duration : 1.8496263027191162 Secs
Generation 25448 Bred
Beginning Natural Selection For Generation : 25448 | Population Size : 75
Generation 25448 High Score : 9 | Duration : 1.5420618057250977 Secs
Generation 25449 Bred
Beginning Natural Selection For Generation : 25449 | Population Size : 75
Generation 

Generation 25493 High Score : 5 | Duration : 2.301480531692505 Secs
Generation 25494 Bred
Beginning Natural Selection For Generation : 25494 | Population Size : 83
Generation 25494 High Score : 7 | Duration : 1.9587435722351074 Secs
Generation 25495 Bred
Beginning Natural Selection For Generation : 25495 | Population Size : 87
Generation 25495 High Score : 7 | Duration : 2.0549252033233643 Secs
Generation 25496 Bred
Beginning Natural Selection For Generation : 25496 | Population Size : 91
Generation 25496 High Score : 8 | Duration : 2.4422993659973145 Secs
Generation 25497 Bred
Beginning Natural Selection For Generation : 25497 | Population Size : 91
Generation 25497 High Score : 11 | Duration : 2.541048049926758 Secs
Generation 25498 Bred
Beginning Natural Selection For Generation : 25498 | Population Size : 91
Generation 25498 High Score : 7 | Duration : 2.7612709999084473 Secs
Generation 25499 Bred
Beginning Natural Selection For Generation : 25499 | Population Size : 95
Generation 

In [73]:
population = load_wrapped_net_population('eigthTrial')[0]

In [18]:
population  = population[:30]
len(population)

30

In [32]:
logic_engine_1 = logic_engine_wrapper(logic_engine=population[3][0])
logic_engine_2 = logic_engine_wrapper(logic_engine=population[5][0])
result = play_game(logic_engines=[logic_engine_1,logic_engine_2],verbose=True,ai_delay=0)

|       |
|       |
|       |
|       |
|       |
|       |
---------
 1234567 
Thinking...
|       |
|       |
|       |
|       |
|       |
|#      |
---------
 1234567 
Thinking...
|       |
|       |
|       |
|       |
|       |
|#     @|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|       |
|#      |
|#     @|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|#      |
|#     @|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|#      |
|#    #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|#      |
|#@   #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|##     |
|#@   #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@      |
|##     |
|#@@  #@|
---------
 1234567 
Thinking...
|       |
|       |
|       |
|@#     |
|##     |
|#@@  #@|
---------
 1234567 
Thinking...
|       |
|       |
|@      |
|@#     |
|##     |
|#@@  #@|
---------
 1234567 


In [14]:
len(generate_standard_net().hidden_layers[0])

42